In [19]:

"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding
more average features and weekly average features on it.

Run 1 Store validation  : .py 1s 045
Run 1 Store submission  : .py 1ss 045
Run all Store validation: .py val 045
Run all Store submission: .py a 045


"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
import lightgbm as lgb
import sys
import math
import gc
import sklearn.metrics as skl_metrics
from sklearn.metrics import mean_squared_error

from nwrmsle_eval import eval_test

# import math
# import sklearn.metrics as skl_metrics
# from sklearn.metrics import mean_squared_error

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)

pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../logs/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

param_1 = "1s"
param_2 = "999"


submit_filename = '../submit/T' + param_2 + '.csv.gz'
val_filename = '../data/V' + param_2 + '.p'

logger.info(submit_filename)
logger.info(val_filename)

if  ((param_1 == "1ss") or (param_1 == "1s")):
    train_out = pd.read_pickle('../data/storeitem_train_1s.p')
    val_out = pd.read_pickle('../data/storeitem_val_1s.p')
    X_test_out = pd.read_pickle('../data/storeitem_test_1s.p')

    item_train_out = pd.read_pickle('../data/item_train_1s.p')
    item_val_out = pd.read_pickle('../data/item_val_1s.p')
    item_X_test_out = pd.read_pickle('../data/item_test_1s.p')

    store_train_out = pd.read_pickle('../data/store_train_1s.p')
    store_val_out = pd.read_pickle('../data/store_val_1s.p')
    store_X_test_out = pd.read_pickle('../data/store_test_1s.p')

    s_f_train_out = pd.read_pickle('../data/storefamily_train_1s.p')
    s_f_val_out = pd.read_pickle('../data/storefamily_val_1s.p')
    s_f_X_test_out = pd.read_pickle('../data/storefamily_test_1s.p')
    
    df_test = pd.read_csv(
        "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
        dtype={'onpromotion': bool},
        parse_dates=["date"]  # , date_parser=parser
    ).set_index(
        ['store_nbr', 'item_nbr', 'date']
    )
    
else:
    train_out = pd.read_pickle('../data/storeitem_train.p')
    val_out = pd.read_pickle('../data/storeitem_val.p')
    X_test_out = pd.read_pickle('../data/storeitem_test.p')

    item_train_out = pd.read_pickle('../data/item_train.p')
    item_val_out = pd.read_pickle('../data/item_val.p')
    item_X_test_out = pd.read_pickle('../data/item_test.p')

    store_train_out = pd.read_pickle('../data/store_train.p')
    store_val_out = pd.read_pickle('../data/store_val.p')
    store_X_test_out = pd.read_pickle('../data/store_test.p')

    s_f_train_out = pd.read_pickle('../data/storefamily_train.p')
    s_f_val_out = pd.read_pickle('../data/storefamily_val.p')
    s_f_X_test_out = pd.read_pickle('../data/storefamily_test.p')

    df_test = pd.read_csv(
        "../input/test.csv", usecols=[0, 1, 2, 3, 4],
        dtype={'onpromotion': bool},
        parse_dates=["date"]  # , date_parser=parser
    ).set_index(
        ['store_nbr', 'item_nbr', 'date']
    )

    
items = pd.read_csv(
    "../input/items.csv",
)

items_val = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items_val = items_val.reindex(val_out['item_nbr'])

logger.info('Load data successful')

###############################################################################
#


2018-01-05 12:09:19,496 __main__ 50 [INFO]    [<module>] start 
2018-01-05 12:09:19,496 __main__ 50 [INFO]    [<module>] start 
2018-01-05 12:09:19,496 __main__ 50 [INFO]    [<module>] start 
2018-01-05 12:09:19,496 __main__ 50 [INFO]    [<module>] start 
2018-01-05 12:09:19,496 __main__ 50 [INFO]    [<module>] start 
2018-01-05 12:09:19,504 __main__ 59 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-05 12:09:19,504 __main__ 59 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-05 12:09:19,504 __main__ 59 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-05 12:09:19,504 __main__ 59 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-05 12:09:19,504 __main__ 59 [INFO]    [<module>] ../submit/T999.csv.gz 
2018-01-05 12:09:19,516 __main__ 60 [INFO]    [<module>] ../data/V999.p 
2018-01-05 12:09:19,516 __main__ 60 [INFO]    [<module>] ../data/V999.p 
2018-01-05 12:09:19,516 __main__ 60 [INFO]    [<module>] ../data/V999.p 
2018-01-05 12:09:19,516 __main__ 60 [INFO]    [<module>] ../d

In [20]:
print(store_train_out.groupby(['date']).size())

date
2016-08-03    2
2016-08-10    2
2016-08-17    2
2016-08-24    2
2017-05-31    2
2017-06-07    2
2017-06-14    2
2017-06-21    2
2017-06-28    2
2017-07-05    2
2017-07-12    2
2017-07-19    2
2017-07-26    2
dtype: int64


In [ ]:


###############################################################################
# Delete index columns before merge 
del train_out["index"]
del item_train_out["index"]
del store_train_out["index"]
del s_f_train_out["index"]

# Merge store-family features
items_c = items.copy()
del items_c["class"], items_c["perishable"]


s_f_train_out = pd.merge(s_f_train_out, items_c, how = 'inner', on=['family'] )
s_f_val_out = pd.merge(s_f_val_out, items_c, how = 'inner', on=['family'] )
s_f_X_test_out = pd.merge(s_f_X_test_out, items_c, how = 'inner', on = ['family'] )

del s_f_train_out['family'], s_f_val_out['family'], s_f_X_test_out['family']

train_out = pd.merge(train_out, s_f_train_out, how='inner', on=['store_nbr','item_nbr','date'])
val_out = pd.merge(val_out, s_f_val_out, how='inner', on=['store_nbr','item_nbr','date'])
X_test_out = pd.merge(X_test_out, s_f_X_test_out, how='inner', on=['store_nbr','item_nbr','date'])

del items_c,s_f_train_out, s_f_val_out, s_f_X_test_out
gc.collect()



# Merge item features 
train_out = pd.merge(train_out, item_train_out, how='inner', on=['item_nbr','date'])
val_out = pd.merge(val_out, item_val_out, how='inner', on=['item_nbr','date'])
X_test_out = pd.merge(X_test_out, item_X_test_out, how='inner', on=['item_nbr','date'])

del item_train_out, item_val_out, item_X_test_out
gc.collect()



# Merge store features
train_out = pd.merge(train_out, store_train_out, how='inner', on=['store_nbr','date'])
val_out = pd.merge(val_out, store_val_out, how='inner', on=['store_nbr','date'])
X_test_out = pd.merge(X_test_out, store_X_test_out, how='inner', on=['store_nbr','date'])

del store_train_out, store_val_out, store_X_test_out
gc.collect()




In [101]:
s_f_train_out = pd.read_pickle('../data/storefamily_train.p')
s_f_train_out.shape

(183365, 39)

In [83]:
train_out = pd.read_pickle('../data/storeitem_train.p')
train_out.shape

(1914275, 144)

In [71]:
# Merge store-family features
items_c = items.copy()
del items_c["class"], items_c["perishable"]

In [86]:
train_out = pd.merge(train_out, items_c, how = 'inner', on=['item_nbr'] )


In [87]:
train_out.shape

(1914275, 145)

In [89]:
train_out[['store_nbr','family','date', 'item_nbr']].head(1)

,store_nbr,family,date,item_nbr
0,1,GROCERY I,2016-08-03,96995


In [103]:
s_f_train_out.groupby(['family']).size()

family
AUTOMOTIVE                     3016
BABY CARE                       234
BEAUTY                         2249
BEVERAGES                     14586
BOOKS                           299
BREAD/BAKERY                  10972
CELEBRATION                    3211
CLEANING                      13832
DAIRY                         13091
DELI                           9724
EGGS                           4134
FROZEN FOODS                   6617
GROCERY I                     14976
GROCERY II                     1794
HARDWARE                        897
HOME AND KITCHEN I             6149
HOME AND KITCHEN II            5395
HOME APPLIANCES                 416
HOME CARE                      9074
LADIESWEAR                     3159
LAWN AND GARDEN                2964
LINGERIE                       2145
LIQUOR,WINE,BEER               6877
MAGAZINES                      2158
MEATS                          5382
PERSONAL CARE                 12246
PET SUPPLIES                   2522
PLAYERS AND ELECTRONI

In [106]:
train_out.groupby(['item_nbr']).size()

item_nbr
96995      286
99197      308
103501     319
103520     594
103665     451
105574     308
105575     363
105576     297
105577     297
105693     594
105737     297
105857     594
106716     594
108079     594
108634     429
108696     583
108698     473
108701     594
108786     594
108797     594
108831     550
108833     297
108862     308
108952     594
111223     594
111397     594
112830     528
114778     561
114790     594
114799     594
          ... 
2116238     55
2116416    572
2118662     55
2120723    121
2121610     55
2121690     99
2122188    506
2122676    187
2122818     11
2122868    176
2122947     44
2123036     77
2123209     33
2123410     99
2123463     22
2123468    286
2123711     44
2123727    176
2123747     22
2123750    198
2123775    154
2123790     77
2123791    132
2123839    121
2123859     33
2123863     77
2124052    528
2126842     44
2126944     55
2127114    253
Length: 4036, dtype: int64

In [74]:
train_out = pd.merge(train_out, s_f_train_out, how='inner', on=['store_nbr','family','date'])

In [80]:
train_out.shape

(68387, 181)

In [79]:
train_out[['store_nbr','family','date', ]]

,store_nbr,family,date
0,45,BREAD/BAKERY,2016-08-03
1,45,BREAD/BAKERY,2016-08-03
2,45,BREAD/BAKERY,2016-08-03
3,45,BREAD/BAKERY,2016-08-03
4,45,BREAD/BAKERY,2016-08-03
5,45,BREAD/BAKERY,2016-08-03
6,45,BREAD/BAKERY,2016-08-03
7,45,BREAD/BAKERY,2016-08-03
8,45,BREAD/BAKERY,2016-08-03
9,45,BREAD/BAKERY,2016-08-03


In [47]:
s_f_train_out = pd.merge(s_f_train_out, items_c, how = 'inner', on = ['family'] )

In [61]:
s_f_train_out.shape

(21047, 39)

In [32]:
train_out.shape

(46241, 213)

In [18]:
train_out.shape

(7800, 248)

In [19]:
logger.info('Preparing traing dataset...')
    
# On validation step, need remove last 2 weeks in the train data
#train_out["date"] = pd.to_datetime(train_out["date"])
#if ((param_1 == "val") or (param_1 == "1s")):
#    train_out = train_out.loc[train_out["date"] < '2017-07-19', ]

    
all_columns = train_out.columns.tolist()

y_columns = ['day'+str(i) for i in range(1, 17)]
x_columns = [item for item in all_columns if item not in y_columns]

features_all = x_columns
features_all.remove("date") 
features_all.remove("item_nbr") 
features_all.remove("store_nbr") 

X_train_out = train_out[x_columns]
X_val_out = val_out[x_columns]

y_train = train_out[y_columns].values
y_val = val_out[y_columns].values

X_train_allF = X_train_out[features_all]
X_val_allF = X_val_out[features_all]
X_test_allF = X_test_out[features_all]

items = items.set_index("item_nbr")
items = items.reindex(train_out.item_nbr)

del train_out
del X_train_out, X_val_out
gc.collect()


2018-01-04 21:15:45,907 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 21:15:45,907 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-04 21:15:45,907 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 


181

In [20]:
X_train_allF.shape

(7800, 229)

In [21]:
##########################################################################
logger.info('Training and predicting models...')

params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []


train_week_2017 = 7
if param_1 != "val":
    train_week_2017 = 9
    
features_all = X_train_allF.columns.tolist()

for i in range(16):
    print("=" * 70)
    logger.info("Step %d" % (i+1))
    print("=" * 70)
    features_t = features_all.copy()

    for j in range(16):
        if j != i:
            features_t.remove('ly_1d_d{}'.format(j))
            features_t.remove('l2y_1d_d{}'.format(j))

    for j in range(7):
        if j != i%7:
            features_t.remove('dow_1_{}_mean'.format(j))
            features_t.remove('dow_4_{}_mean'.format(j))
            features_t.remove('dow_8_{}_mean'.format(j))
            features_t.remove('dow_13_{}_mean'.format(j))
            features_t.remove('dow_26_{}_mean'.format(j))
            features_t.remove('dow_52_{}_mean'.format(j))
            features_t.remove('dow_ly3w_{}_mean'.format(j))
            features_t.remove('dow_ly8w_{}_mean'.format(j))
            
            features_t.remove('item_dow_4_{}_mean'.format(j))
            features_t.remove('item_dow_13_{}_mean'.format(j))
            features_t.remove('item_dow_26_{}_mean'.format(j))
            features_t.remove('item_dow_52_{}_mean'.format(j))

            features_t.remove('store_dow_4_{}_mean'.format(j))
            features_t.remove('store_dow_13_{}_mean'.format(j))
            features_t.remove('store_dow_26_{}_mean'.format(j))
            features_t.remove('store_dow_52_{}_mean'.format(j))     

            
            features_t.remove('s_f_dow_4_{}_mean'.format(j))
            features_t.remove('s_f_dow_13_{}_mean'.format(j))
            features_t.remove('s_f_dow_26_{}_mean'.format(j))
            features_t.remove('s_f_dow_52_{}_mean'.format(j))
            
    X_train = X_train_allF[features_t]
    X_val = X_val_allF[features_t]
    X_test = X_test_allF[features_t]

    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]]) * 0.25 + 1
    )

    if ((param_1 == "val") or (param_1 == "1s")):
        dval = lgb.Dataset(
             X_val, label=y_val[:, i], reference=dtrain,
             weight=items_val["perishable"] * 0.25 + 1,
             categorical_feature=cate_vars)

        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain, dval], early_stopping_rounds=50,
            verbose_eval=100
        )

    else:

        bst = lgb.train(
            params, dtrain, num_boost_round=MAX_ROUNDS,
            valid_sets=[dtrain], early_stopping_rounds=50, verbose_eval=100
        )

    logger.info("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))

    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

    if ((param_1 == "val") or (param_1 == "1s")):
        val_pred.append(bst.predict(
            X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))


2018-01-04 21:15:46,231 __main__ 2 [INFO]    [<module>] Training and predicting models... 
2018-01-04 21:15:46,231 __main__ 2 [INFO]    [<module>] Training and predicting models... 
2018-01-04 21:15:46,231 __main__ 2 [INFO]    [<module>] Training and predicting models... 
2018-01-04 21:15:46,236 __main__ 30 [INFO]    [<module>] Step 1 
2018-01-04 21:15:46,236 __main__ 30 [INFO]    [<module>] Step 1 
2018-01-04 21:15:46,236 __main__ 30 [INFO]    [<module>] Step 1 
/home/zyp/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.160178
[200]	training's l2: 0.14078
[300]	training's l2: 0.126444
[400]	training's l2: 0.115681
[500]	training's l2: 0.106225


2018-01-04 21:15:47,849 __main__ 97 [INFO]    [<module>] promo_0: 36565.68
mean_7_2017: 9201.66
mean_60_2017: 2634.81
mean_3_2017: 1839.70
dow_26_0_mean: 1244.54
day_1_2017: 964.78
mean_30_2017: 948.97
dow_13_0_mean: 925.76
mean_21_2017: 884.80
dow_4_0_mean: 530.31
mean_14_2017: 511.37
dow_1_0_mean: 489.77
item_mean_21_2017: 406.18
dow_8_0_mean: 357.98
dow_ly8w_0_mean: 272.26
mean_364_2017: 265.95
mean_42_2017: 244.73
mean_91_2017: 212.70
promo_140_2017: 197.88
dow_52_0_mean: 193.24
mean_182_2017: 175.17
item_mean_7_2017: 165.19
ly_1d_d0: 164.44
mean_ly_14_2017: 141.40
item_day_1_2017: 129.22
item_dow_4_0_mean: 127.20
mean_ly_n16d_2017: 126.49
mean_ly_7_2017: 125.02
item_dow_52_0_mean: 120.38
mean_ly_21_2017: 117.51
item_dow_13_0_mean: 114.03
item_mean_91_2017: 110.86
item_dow_26_0_mean: 107.39
mean_ly_30_2017: 104.51
item_mean_42_2017: 104.33
item_mean_364_2017: 103.49
mean_140_2017: 102.40
promo_60_2017: 89.22
dow_ly3w_0_mean: 83.66
item_mean_182_2017: 73.21
l2y_1d_d0: 70.75
promo_14

Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.242505
[200]	training's l2: 0.215537
[300]	training's l2: 0.194587
[400]	training's l2: 0.177105


2018-01-04 21:15:49,503 __main__ 97 [INFO]    [<module>] mean_7_2017: 17060.20
mean_21_2017: 7562.23
mean_14_2017: 3288.28
dow_13_1_mean: 1443.71
dow_26_1_mean: 1259.69
mean_30_2017: 1171.28
dow_8_1_mean: 1026.69
promo_0: 1021.17
mean_3_2017: 965.91
mean_42_2017: 702.92
dow_52_1_mean: 544.30
mean_ly_7_2017: 429.03
dow_1_1_mean: 366.35
item_mean_7_2017: 311.91
dow_4_1_mean: 305.24
mean_60_2017: 264.06
ly_1d_d1: 260.22
mean_182_2017: 242.00
item_day_1_2017: 228.01
day_1_2017: 215.25
item_mean_42_2017: 195.92
dow_ly3w_1_mean: 193.75
item_mean_21_2017: 188.38
mean_ly_14_2017: 174.25
item_mean_364_2017: 166.81
mean_140_2017: 164.21
mean_91_2017: 162.80
item_dow_4_1_mean: 161.38
mean_364_2017: 160.94
mean_ly_n16d_2017: 154.68
dow_ly8w_1_mean: 152.89
mean_ly_30_2017: 149.64
promo_140_2017: 146.03
l2y_1d_d1: 141.46
item_mean_91_2017: 141.25
item_dow_13_1_mean: 130.67
item_dow_26_1_mean: 127.42
item_mean_182_2017: 120.08
mean_ly_21_2017: 119.31
item_dow_52_1_mean: 112.59
store_mean_182_2017: 94

...

[500]	training's l2: 0.165674
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.177368
[200]	training's l2: 0.153292
[300]	training's l2: 0.137384
[400]	training's l2: 0.125022


2018-01-04 21:16:06,634 __main__ 97 [INFO]    [<module>] promo_14: 39974.87
mean_21_2017: 3075.19
mean_182_2017: 3062.60
mean_14_2017: 2425.56
dow_26_0_mean: 1385.71
mean_7_2017: 1136.88
dow_13_0_mean: 1107.46
mean_140_2017: 970.47
mean_30_2017: 780.08
dow_52_0_mean: 415.03
ly_1d_d14: 393.70
mean_42_2017: 391.04
dow_1_0_mean: 343.28
dow_ly8w_0_mean: 313.58
mean_91_2017: 286.92
item_mean_7_2017: 277.38
mean_3_2017: 273.51
day_1_2017: 270.82
dow_ly3w_0_mean: 240.95
item_mean_91_2017: 240.28
mean_364_2017: 236.12
promo_140_2017: 217.40
dow_8_0_mean: 208.98
mean_ly_n16d_2017: 208.94
item_mean_364_2017: 188.98
item_mean_21_2017: 187.47
mean_60_2017: 168.89
item_day_1_2017: 168.39
item_dow_26_0_mean: 162.10
item_dow_4_0_mean: 156.04
dow_4_0_mean: 152.02
item_dow_13_0_mean: 148.81
item_mean_42_2017: 131.84
mean_ly_30_2017: 131.25
item_dow_52_0_mean: 127.81
l2y_1d_d14: 120.55
item_mean_182_2017: 105.32
mean_ly_21_2017: 103.95
mean_ly_7_2017: 101.85
mean_ly_14_2017: 92.62
promo_60_2017: 91.56
s

[500]	training's l2: 0.11497
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.246759
[200]	training's l2: 0.217158
[300]	training's l2: 0.195827
[400]	training's l2: 0.178652


2018-01-04 21:16:08,114 __main__ 97 [INFO]    [<module>] mean_42_2017: 17692.41
promo_14: 3665.93
mean_7_2017: 3535.22
dow_8_1_mean: 2238.92
mean_30_2017: 1511.73
mean_14_2017: 1419.53
dow_26_1_mean: 1215.06
mean_91_2017: 1097.75
dow_52_1_mean: 796.99
mean_182_2017: 520.99
dow_13_1_mean: 509.25
mean_140_2017: 502.17
mean_21_2017: 458.35
dow_ly3w_1_mean: 451.32
dow_ly8w_1_mean: 296.92
ly_1d_d15: 286.88
dow_1_1_mean: 277.14
mean_60_2017: 262.11
item_mean_7_2017: 248.36
item_dow_13_1_mean: 215.51
item_mean_21_2017: 214.48
mean_3_2017: 213.72
item_day_1_2017: 211.52
mean_ly_14_2017: 208.14
item_mean_42_2017: 207.39
item_mean_364_2017: 205.36
item_dow_26_1_mean: 192.79
item_dow_52_1_mean: 180.64
day_1_2017: 176.72
item_dow_4_1_mean: 169.30
dow_4_1_mean: 167.40
mean_ly_7_2017: 166.08
mean_ly_n16d_2017: 161.32
item_mean_91_2017: 160.39
l2y_1d_d15: 155.55
item_mean_182_2017: 154.97
mean_ly_30_2017: 148.25
mean_364_2017: 141.90
mean_ly_21_2017: 132.50
promo_140_2017: 118.67
promo_60_2017: 79.46

[500]	training's l2: 0.164311
